In [1]:
!git clone https://github.com/lukun199/SemanticRL.git

Cloning into 'SemanticRL'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 158 (delta 40), reused 114 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (158/158), 461.14 KiB | 3.12 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [2]:
%cd /kaggle/working/SemanticRL

/kaggle/working/SemanticRL


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 26.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement numpy==1.21.6 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 1.13.3, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.19.0, 1.19.1, 1.19.2, 1.19.3, 1.19.4, 

In [4]:
!cp -r /kaggle/input/european-parliament/dataset /kaggle/working/SemanticRL

In [5]:
%cd /kaggle/working/SemanticRL/dataset

/kaggle/working/SemanticRL/dataset


In [6]:
!tar -zxvf fr-en.tgz 

europarl-v7.fr-en.en
europarl-v7.fr-en.fr


In [7]:
!python3 /kaggle/working/SemanticRL/preprocess_captions.py --data_root /kaggle/working/SemanticRL/dataset

English data: sentences=2007723, min=0, max=668
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
Saved: english.pkl
resumption of the session
i declare resumed the session of the european parliament adjourned on friday december and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period
although as you will have seen the dreaded millennium bug failed to materialise still the people in a number of countries suffered a series of natural disasters that truly were dreadful
you have requested a debate on this subject in the course of the next few days during this partsession
in the meantime i should like to observe a minute s silence as a number of members have requested on behalf of all the victims concerned particularly those of the terrible storms in the various countries of the european union
please rise then for this minute s silenc

### Training SemanticRL-JSCC

In [12]:
%%writefile /kaggle/working/SemanticRL/utils.py
"""
lukun199@gmail.com
3rd Feb., 2021

# utils.py
"""
import os, sys, time, glob, shutil
import torch
from torch.distributions import Normal

# Communication Utils: Channel Model, error rate, etc.

class Normlize_tx:
    def __init__(self, _iscomplex):
        self._iscomplex = _iscomplex
    def apply(self, _input):
        _dim = _input.shape[1]//2 if self._iscomplex else _input.shape[1]
        _norm = _dim**0.5 / torch.sqrt(torch.sum(_input**2, dim=1))
        return _input*_norm.view(-1,1)

class Channel:
    # returns the message when passed through a channel.
    # AGWN, Fading
    # Note that we need to make sure that the colle map will not change in this
    # step, thus we should not use *= and +=.
    def __init__(self, _iscomplex):
        self._iscomplex = _iscomplex

    def ideal_channel(self, _input):
        return _input

    def awgn(self, _input, _snr):
        _std = (10**(-_snr/10.)/2)**0.5 if self._iscomplex else (10**(-_snr/10.))**0.5  # for complex signals.
        _input = _input + torch.randn_like(_input) * _std
        #print(_std)
        return _input

    def awgn_physical_layer(self, _input, _snr):
        _std = (10**(-_snr/10.)/2)**0.5
        _input = _input + torch.randn_like(_input) * _std
        #print(_std)
        return _input

    def fading(self, _input, _snr):
        # ref from DeepJSCC-f https://github.com/kurka/deepJSCC-feedback
        if self._iscomplex:
            _shape = _input.shape
            _dim = _shape[1]//2
            _std = (10**(-_snr/10.)/2)**0.5
            _mul = torch.abs(torch.randn(_shape[0], 2)/(2**0.5))  # should divide 2**0.5 here.
            _input_ = _input.clone()
            _input_[:,:_dim] *= _mul[:,0].view(-1,1)
            _input_[:,_dim:] *= _mul[:,1].view(-1,1)
            _input = _input_
        else:
            _std = (10**(-_snr/10.))**0.5
            _input = _input * torch.abs(torch.randn(_input.shape[0], 1)).to(_input)
        _input = _input + torch.randn_like(_input) * _std
        #print(_std)
        return _input

    def phase_invariant_fading(self, _input, _snr):
        # ref from DeepJSCC-f
        _std = (10**(-_snr/10.)/2)**0.5 if self._iscomplex else (10**(-_snr/10.))**0.5
        if self._iscomplex:
            _mul = (torch.randn(_input.shape[0], 1)**2/2. + torch.randn(_input.shape[0], 1)**2/2.)**0.5
        else:
            _mul = (torch.randn(_input.shape[0], 1)**2 + torch.randn(_input.shape[0], 1)**2) ** 0.5
        _input = _input * _mul.to(_input)
        _input = _input +  torch.randn_like(_input) * _std
        #print(_std)
        return _input

# Other Utils:

class Crit:

    def __call__(self, mode, *args):
        return getattr(self, '_' + mode)(*args)

    def _ce(self, pred, target, lengths):
        mask = pred.new_zeros(len(lengths), target.size(1))
        for i, l in enumerate(lengths): # length can be bigger than the true len
            mask[i, :l] = 1

        loss = - pred.gather(2, target.unsqueeze(2)).squeeze(2) * mask   # log counted.
        loss = torch.sum(loss) / torch.sum(mask)  # ln(vocab_dim=24064) is around 10
        return loss

    def _rl(self, seq_logprobs, seq_masks, reward):
        # seq_logprobs[bs, T] reward:[bs, 1] or [bs, T]
        output = - seq_logprobs * seq_masks * reward
        output = torch.sum(output) / torch.sum(seq_masks)
        return output

    def _tx_gaussian_sample(self, log_samples, reward):
        return -(log_samples*reward).mean()

# SCSIU
class GaussianPolicy():

    def forward(self, x, std=0.1):
        return x + torch.randn_like(x) * std

    def forward_sample(self, mean, std=0.1):
        dist = Normal(mean, std)
        action = dist.sample()
        ln_prob = dist.log_prob(action)
        return action, ln_prob

# others

def clip_gradient(optimizer, grad_clip):
    for group in optimizer.param_groups:
        for param in group['params']:
            param.grad.data.clamp_(-grad_clip, grad_clip)

def time_consum_wrapper(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        print(func.__name__, 'is running')
        res = func(*args, **kwargs)
        print("time func takes", time.time() - start)
        return res
    return wrapper


def smaple_n_times(n, x):
        if n>1:
            x = x.unsqueeze(1) # Bx1x...
            x = x.expand(-1, n, *([-1]*len(x.shape[2:])))
            x = x.reshape(x.shape[0]*n, *x.shape[2:])
        return x

def copyStage1ckpts(frompath, topath, strs='resume_from_ce_'):
    os.makedirs(topath, exist_ok=True)
    files = glob.glob(os.path.join(frompath, '*24.pth'))
    for file in files:
        shutil.copyfile(file, os.path.join(topath, strs+os.path.basename(file)))


if __name__ =='__main__':

    is_complex = False
    n = Normlize_tx(is_complex)
    x = torch.tensor([[1., 2., 3., 4., 5., 6., 7., 8., 9., 10.], [18., 2., 3., 4., 5., 6., 7., 8., 9., 10.]])
    y = n.apply(x)
    print(y)
    #for i in range(x.shape[1]//2):
    #    print(y[:,i], y[:,5+i])

    c = Channel(is_complex)
    # x = torch.ones(2,4)
    z = c.awgn(y,10)
    print(z)

Overwriting /kaggle/working/SemanticRL/utils.py


In [9]:
%%writefile /kaggle/working/SemanticRL/config/config_AWGN_CE.yaml
snr: 10
multiple_sample: 5
scheduled_sampling_start: 18
iscomplex: 1
channel_type: 'awgn'
batch_size: 64
channel_dim: 256
ckpt_resume: -1
lr_milestones: '20 88'
max_epoch: 25
seeds: 7
init_learning_rate: 1e-3
save_model_path: './ckpt_AWGN_CE/'
RL_training: 0
device: 'cuda:0'
backbone: 'LSTM'

Overwriting /kaggle/working/SemanticRL/config/config_AWGN_CE.yaml


In [10]:
%%writefile /kaggle/working/SemanticRL/config/config_AWGN_CE_Stage2.yaml
snr: 10
multiple_sample: 5
scheduled_sampling_start: 18
iscomplex: 1
channel_type: 'awgn'
batch_size: 64
channel_dim: 256
ckpt_resume: 25
lr_milestones: '160'
max_epoch: 75
seeds: 7
init_learning_rate: 1e-4
save_model_path: './ckpt_AWGN_CE_Stage2/'
RL_training: 0
device: 'cuda:0'
backbone: 'LSTM'

Overwriting /kaggle/working/SemanticRL/config/config_AWGN_CE_Stage2.yaml


In [11]:
import os
print(os.getcwd())

/kaggle/working/SemanticRL/dataset


In [16]:
%cd /kaggle/working/SemanticRL

/kaggle/working/SemanticRL


In [13]:
!cp -r /kaggle/input/ckpt_awgn_ce/pytorch/default/1 /kaggle/working/SemanticRL

In [14]:
import os
old_folder_path = "/kaggle/working/SemanticRL/1"
new_folder_path = "/kaggle/working/SemanticRL/ckpt_AWGN_CE"
os.rename(old_folder_path, new_folder_path)

In [13]:
%%writefile /kaggle/working/SemanticRL/self_critical/utils.py
import numpy as np
import torch
import torch.nn as nn
import tqdm
import os
from .cider.pyciderevalcap.ciderD.ciderD import CiderD
from .bleu.bleu import Bleu


def write2txt(fp, info, mode='a'):
    """
    Write information to a text file
    
    Args:
        fp: file path
        info: information to write
        mode: file mode ('a' for append, 'w' for write)
    """
    with open(fp, mode=mode) as f:
        f.write(info)
        f.write('\n')


def _array_to_str(arr, sos_token, eos_token):
    out = ''
    for i in range(len(arr)):
        if arr[i] == eos_token:
            break
        out += str(arr[i]) + ' '
    out += str(eos_token)  # optional
    return out.strip()


def get_4_bleu_score(output, ground_truth, reward_scorer, sos_token=1, eos_token=2):
    """
    Calculate BLEU-1,2,3,4 scores for CE baseline evaluation
    
    Args:
        output: decoder output predictions [batch_size, seq_len, vocab_size]
        ground_truth: ground truth sentences [batch_size, seq_len]
        reward_scorer: BLEU scorer instance
        sos_token: start of sentence token (default=1)
        eos_token: end of sentence token (default=2)
    
    Returns:
        tuple: (bleu1_score, bleu2_score, bleu3_score, bleu4_score)
    """
    device = output.device
    batch_size = output.size(0)
    
    # Convert output probabilities to predicted tokens
    predicted_tokens = torch.argmax(output, dim=-1)  # [batch_size, seq_len]
    
    # Convert to numpy for evaluation
    predicted_tokens = predicted_tokens.cpu().numpy()
    ground_truth = ground_truth.cpu().numpy()
    
    # Prepare data for BLEU calculation
    sample_result = []
    gts = {}
    
    for i in range(batch_size):
        # Convert predicted tokens to string
        pred_str = _array_to_str(predicted_tokens[i], sos_token, eos_token)
        sample_result.append({'image_id': i, 'caption': [pred_str]})
        
        # Convert ground truth to string
        gt_str = _array_to_str(ground_truth[i], sos_token, eos_token)
        gts[i] = [gt_str]
    
    # Calculate BLEU scores using the reward_scorer
    if isinstance(reward_scorer, Bleu):
        _, scores_mat = reward_scorer.compute_score(gts, sample_result)
        
        # Extract individual BLEU scores
        scores_b1 = np.array(scores_mat[0]).mean()
        scores_b2 = np.array(scores_mat[1]).mean()
        scores_b3 = np.array(scores_mat[2]).mean()
        scores_b4 = np.array(scores_mat[3]).mean()
        
        return scores_b1, scores_b2, scores_b3, scores_b4
    else:
        print("Warning: reward_scorer is not a BLEU scorer")
        return 0.0, 0.0, 0.0, 0.0


def log_training_metrics(epoch, loss, bleu1, bleu2, bleu3, bleu4, log_path):
    """
    Log training metrics to text file
    
    Args:
        epoch: current epoch
        loss: training loss
        bleu1, bleu2, bleu3, bleu4: BLEU scores
        log_path: path to log file
    """
    # Create directory if it doesn't exist
    log_dir = os.path.dirname(log_path)
    if log_dir and not os.path.exists(log_dir):
        os.makedirs(log_dir, exist_ok=True)
    
    info = f'epoch:{epoch} loss:{loss:.6f} bleu1:{bleu1:.4f} bleu2:{bleu2:.4f} bleu3:{bleu3:.4f} bleu4:{bleu4:.4f}'
    write2txt(log_path, info)


def get_ciderd_scorer_europarl(split_captions, test_data_num, sos_token, eos_token):

    all_caps = np.concatenate((split_captions, test_data_num))
    print('====> get_ciderd_scorer begin, seeing {} sentences'.format(len(all_caps)))

    refs_idxs = []
    for caps in all_caps:
        ref_idxs = []
        ref_idxs.append(_array_to_str(caps, sos_token, eos_token))
        refs_idxs.append(ref_idxs)

    scorer = CiderD(refs_idxs)
    del refs_idxs
    del ref_idxs
    print('====> get_ciderd_scorer end')
    return scorer

def get_bleu_scorer_europarl(n=4):

    scorer = Bleu(n=n)
    print('====> get_bleu_scorer end')

    return scorer


def get_self_critical_reward_sc(sample_captions, fns, ground_truth,
                             sos_token, eos_token, scorer):
    # the first dim of fns are the same with samples. fns is a list.
    device = sample_captions.device
    batch_size = len(ground_truth)
    seq_per_img = len(fns) // batch_size
    sample_captions = sample_captions.cpu().numpy()
    ground_truth = ground_truth.cpu().numpy()

    max_seq_len = sample_captions.shape[1]
    sample_result = []
    gts = {}
    # first multiple samples.
    for fn in fns:
        sample_result.append({'image_id': fn, 'caption': [_array_to_str(sample_captions[fn], sos_token, eos_token)]})
        caps = []
        caps.append(_array_to_str(ground_truth[fn//seq_per_img][:max_seq_len], sos_token, eos_token))
        gts[fn] = caps

    if isinstance(scorer, CiderD):
        _, scores = scorer.compute_score(gts, sample_result)  # [bs*5,1]
        scores = torch.from_numpy(scores).to(device).view(-1, seq_per_img)  # [bs,5]
        detailed_reward = None
    elif isinstance(scorer, Bleu):
        _, scores_mat = scorer.compute_score(gts, sample_result)
        scores_b1 = np.array(scores_mat[0]).mean()
        scores_b2 = np.array(scores_mat[1]).mean()
        scores_b3 = np.array(scores_mat[2]).mean()
        scores_b4 = np.array(scores_mat[3]).mean()
        detailed_reward = (scores_b1, scores_b2, scores_b3, scores_b4)
        scores = (np.array(scores_mat[0]) + np.array(scores_mat[3]))/2
        scores = torch.from_numpy(scores).to(device).view(-1, seq_per_img)  # [bs,5]

    scores.requires_grad = False
    baseline = (scores.sum(1, keepdim=True) - scores) / (scores.shape[1] - 1) # [bs,5]
    scores = scores - baseline # [bs,5]
    scores = scores.view(-1, 1)  # [bs*5, 1]
    return scores, baseline.mean(), detailed_reward


def get_self_critical_reward_newsc_TXRL(sample_captions, fns, ground_truth,
                                        sos_token, eos_token, scorer):
    # the first dim of fns are the same with samples. fns is a list.
    device = sample_captions.device
    batch_size = len(ground_truth)
    seq_per_img = len(fns) // batch_size
    sample_captions = sample_captions.cpu().numpy()
    ground_truth = ground_truth.cpu().numpy()

    max_seq_len = sample_captions.shape[1]
    sample_result = []
    gts = {}
    # first multiple samples.
    for fn in fns:
        sample_result.append({'image_id': fn, 'caption': [_array_to_str(sample_captions[fn], sos_token, eos_token)]})
        caps = []
        caps.append(_array_to_str(ground_truth[fn // seq_per_img][:max_seq_len], sos_token, eos_token))
        gts[fn] = caps

    _, scores = scorer.compute_score(gts, sample_result)  # [bs*5,1]
    scores = torch.from_numpy(scores).to(device).view(-1, seq_per_img)  # [bs,5]
    scores.requires_grad = False
    if seq_per_img > 1:
        baseline = (scores.sum(1, keepdim=True) - scores) / (scores.shape[1] - 1)  # [bs,5]
        scores = scores - baseline  # [bs,5]
    scores = scores.view(-1, 1)  # [bs*5, 1]

    if seq_per_img > 1:
        return scores, baseline.mean()
    else:
        return scores, scores.mean()

Overwriting /kaggle/working/SemanticRL/self_critical/utils.py


In [14]:
%%writefile /kaggle/working/SemanticRL/Trainng_SemanticRL.py
"""
lukun199@gmail.com
18th Feb., 2021

# train.py
"""
import os, argparse, yaml, random
import platform
import re

import numpy as np
import torch
import torch.utils.data as data
import torch.optim as optim
from data_loader import Dataset_sentence, collate_func
from model import get_model
from utils import Normlize_tx, Channel, Crit, clip_gradient, copyStage1ckpts, smaple_n_times, GaussianPolicy
from self_critical.utils import get_ciderd_scorer_europarl, get_bleu_scorer_europarl, get_self_critical_reward_sc, get_self_critical_reward_newsc_TXRL, get_4_bleu_score, log_training_metrics

parser = argparse.ArgumentParser()
parser.add_argument("--snr", type=int, default=10)
parser.add_argument("--multiple_sample", type=int, default=5)  # number of parallel samples
parser.add_argument("--scheduled_sampling_start", type=int, default=18)  # when to start scheduled sampling
parser.add_argument("--iscomplex", type=float, default=1)
parser.add_argument("--channel_type", type=str, default='awgn')
parser.add_argument("--batch_size", type=int, default=64)
parser.add_argument("--channel_dim", type=int, default=256)
parser.add_argument("--ckpt_resume", type=int, default=-1)
parser.add_argument("--lr_milestones", type=str, default='160') # update the learning rate
parser.add_argument("--max_epoch", type=int, default=202)
parser.add_argument("--seeds", type=int, default=7)  # not used in the paper. but we can assign it
parser.add_argument("--init_learning_rate", type=float, default=0.0001)
parser.add_argument("--save_model_path", type=str, default="./ckpt_RL/")
parser.add_argument("--dataset_path", type=str, default="")
parser.add_argument("--RL_training", type=float, default=1)
parser.add_argument("--reward_type", type=str, default='CIDEr')  # BLEU, CIDEr
parser.add_argument("--device", type=str, default="cuda:0")
parser.add_argument("--backbone", type=str, default="LSTM")  # 'Transformer' 'LSTM'
parser.add_argument("--training_config", type=str, default="")
parser.add_argument("--teacher_forcing", type=float, default=0)
parser.add_argument("--SemanticRL_JSCC", type=float, default=1)  # set 0 for SemanticRL-SCSIU
parser.add_argument("--accumulate_grad", type=float, default=20)  # for variant SCSIU
parser.add_argument("--log_bleu", action='store_true', help='log BLEU scores during CE training')
parser.add_argument("--log_file", type=str, default="./results/training_log.txt", help='path to log file')
args = parser.parse_args()
if args.training_config:
    f = yaml.safe_load(open(args.training_config, 'r'))
    for kk, vv in f.items():
        setattr(args, kk, vv)
setattr(args, 'init_learning_rate', float(args.init_learning_rate))
assert args.init_learning_rate<1, 'please check if your learning rate < 1'


# seeds
def init_seeds(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
init_seeds(args.seeds)

print('\n[*]---------------args init done')
print('args preview:')
for k in args.__dict__:  print(k + ": " + str(args.__dict__[k]))
print('[*]---------------starting preparing dataset and network. Preparing the dataset may take a few minutes')

os.makedirs(args.save_model_path, exist_ok=True)
device = torch.device(args.device)
opt_decoder = 1
train_loader_params = {'batch_size': args.batch_size,
                       'shuffle': True, 'num_workers':4,  # set to 0 in Windows system
                       'collate_fn': lambda x: collate_func(x),
                       'drop_last': True}  # setting as False should also be OK
data_train = Dataset_sentence(_path = args.dataset_path, use_sos=args.backbone=='Transformer')
train_data_loader = data.DataLoader(data_train,**train_loader_params)


embeds_shared = get_model('Embeds')(vocab_size = data_train.get_dict_len(), num_hidden=128).to(device)
encoder = get_model(args.backbone+'Encoder')(channel_dim=args.channel_dim, embedds=embeds_shared).to(device)
decoder = get_model(args.backbone+'Decoder')(channel_dim=args.channel_dim, embedds=embeds_shared,
                             vocab_size=data_train.get_dict_len()).to(device)
normlize_layer = Normlize_tx(_iscomplex=args.iscomplex)
channel = Channel(_iscomplex=args.iscomplex)
policy = GaussianPolicy()

# print #params             a+b-c since embeds_shared is contained in both TX and RX
nums_model = sum(x.numel() for x in encoder.parameters() if x.requires_grad is True) + \
             sum(x.numel() for x in encoder.parameters() if x.requires_grad is True) - \
             sum(x.numel() for x in embeds_shared.parameters() if x.requires_grad is True)
print("Model {} have {} paramerters in total".format(args.backbone, nums_model))

# load saved ckpt
if args.ckpt_resume>0:
    copyStage1ckpts('./ckpt_{}_CE'.format('AWGN' if args.channel_type=='awgn' else 'FIF'), args.save_model_path)
    embeds_shared.load_state_dict(
        torch.load(args.save_model_path + 'resume_from_ce_embeds_shared_epoch{}.pth'.format(args.ckpt_resume - 1)))
    encoder.load_state_dict(torch.load(args.save_model_path + 'resume_from_ce_encoder_epoch{}.pth'.format(args.ckpt_resume-1)))
    decoder.load_state_dict(torch.load(args.save_model_path + 'resume_from_ce_decoder_epoch{}.pth'.format(args.ckpt_resume-1)))
    print('[*]---------------loaded ckpt at' + args.save_model_path)


# multigpu training with ddp brings a performance degradation. this is perhaps caused by the random number in the channel.
# so we will not provide the ddp version

_params = list(list(embeds_shared.parameters()) + list(decoder.parameters()) + list(encoder.parameters()))
if args.SemanticRL_JSCC:
    optimizer = torch.optim.Adam(_params, lr=args.init_learning_rate)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                                           milestones=[int(x) for x in args.lr_milestones.split(' ')],
                                           gamma=0.5)
else:  # SemanticRL-SCSIU
    # we assign the shared embedding to decoder. this is not a must.
    optimizer_encoder = torch.optim.Adam(list(set(list(encoder.parameters())) - set(list(embeds_shared.parameters()))),
                                         lr=args.init_learning_rate)
    optimizer_decoder = torch.optim.Adam(decoder.parameters(), lr=args.init_learning_rate)
    scheduler_encoder = optim.lr_scheduler.MultiStepLR(optimizer_encoder,
                                                       milestones=[int(x) for x in args.lr_milestones.split(' ')],
                                                       gamma=0.5)
    scheduler_decoder = optim.lr_scheduler.MultiStepLR(optimizer_decoder,
                                                       milestones=[int(x) for x in args.lr_milestones.split(' ')],
                                                       gamma=0.5)


# loss function
crit = Crit()
print('[*]---------------network config done.')

# reward config for RL training
if args.RL_training:
    reward_scorer = get_ciderd_scorer_europarl(data_train.data_num.numpy(), np.array(data_train.test_data_num),
                                               sos_token=1, eos_token=2) if args.reward_type == 'CIDEr' \
                    else get_bleu_scorer_europarl()
else:
    # For CE baseline, we also need BLEU scorer for evaluation
    reward_scorer = get_bleu_scorer_europarl()


def train(encoder, decoder, device, train_loader, optimizer, epoch):

    # set model as training mode
    embeds_shared.train()
    encoder.train()
    decoder.train()

    print('--------------------epoch: %d' % epoch)

    # scheduled sampling
    frac = (epoch - args.scheduled_sampling_start) // 7  # hyper params. you can set to others
    ss_prob = min(0.05 * frac, 0.25)
    if ss_prob<0: ss_prob=0  # avoid ss_prob<0

    # training loops
    total_loss = 0
    total_bleu1, total_bleu2, total_bleu3, total_bleu4 = 0, 0, 0, 0
    num_batches = 0
    
    for batch_idx, (train_sents, len_batch) in enumerate(train_loader):
        # Note, for LSTM we feed m=[w_1, w_2, w_3, ...]
        # for Transformer we feed m=[<SOS>, w_1, w_2, w_3, ...]

        # distribute data to device
        train_sents = train_sents.to(device)
        optimizer.zero_grad()
        output, src_mask = encoder(train_sents, len_batch)  # for both LSTM and Transformer
        output = normlize_layer.apply(output)
        output = getattr(channel, args.channel_type)(output, _snr=args.snr)
        if not args.RL_training:
            output = decoder.forward_ce(output, train_sents, src_mask, ss_prob)
            loss = crit('ce', output, train_sents if 'LSTM' in args.backbone else train_sents[:,1:],
                        len_batch if 'LSTM' in args.backbone else [x-1 for x in len_batch])  # remove sos.
            
            # Calculate BLEU scores for CE baseline if logging is enabled
            if (args.log_bleu or not args.RL_training) and batch_idx % 100 == 0:  # Calculate BLEU every 100 batches to save time
                try:
                    bleu1, bleu2, bleu3, bleu4 = get_4_bleu_score(
                        output, 
                        train_sents if 'LSTM' in args.backbone else train_sents[:,1:],
                        reward_scorer
                    )
                    total_bleu1 += bleu1
                    total_bleu2 += bleu2
                    total_bleu3 += bleu3
                    total_bleu4 += bleu4
                    num_batches += 1
                except Exception as e:
                    print(f"Error calculating BLEU: {e}")
        else:
            if args.teacher_forcing>0:
                sample_captions, sample_logprobs, seq_masks = decoder.forward_rl_ssprob(output, train_sents,
                                 sample_max=False, multiple_sample=args.multiple_sample,  x_mask=src_mask)

            else:
                # decoder.sample_max_batch(output, src_mask) # for inference only
                sample_captions, sample_logprobs, seq_masks = decoder.forward_rl(output, sample_max=False,
                                                     multiple_sample=args.multiple_sample, x_mask=src_mask)
            fns = list(range(sample_logprobs.size()[0]))
            advantage, reward_mean, detailed_reward = get_self_critical_reward_sc(sample_captions,
                fns, train_sents if 'LSTM' in args.backbone else train_sents[:,1:], 1, 2, reward_scorer)
            loss = crit('rl', sample_logprobs, seq_masks, advantage)

        # When using Transformer backbone, it is recommended to set args.set teacher_forcing >0, and combine CE and RL loss to 
        # get the best performance. i.e., `loss = loss_ce*lambda + loss_rl` where lambda can be, say, 0.2.
        loss.backward()
        clip_gradient(optimizer, 0.1)
        optimizer.step()
        
        total_loss += loss.item()

        if batch_idx%500==0:
            if not args.RL_training:
                print('[%4d / %4d]    '%(batch_idx, epoch) , '    loss = ', loss.item())
            else:
                print('[%4d / %4d]    ' % (batch_idx, epoch), ' advantage_mean =%.4f,   loss = %.4f,   train_reward = %.4f'
                      % (float(advantage.mean()), loss.item(), float(reward_mean)), detailed_reward)

    # Calculate average loss and BLEU scores for the epoch
    avg_loss = total_loss / len(train_loader)
    avg_bleu1 = total_bleu1 / max(num_batches, 1)
    avg_bleu2 = total_bleu2 / max(num_batches, 1)
    avg_bleu3 = total_bleu3 / max(num_batches, 1)
    avg_bleu4 = total_bleu4 / max(num_batches, 1)
    
    # Log metrics if enabled
    if (args.log_bleu or not args.RL_training):
        log_training_metrics(epoch, avg_loss, avg_bleu1, avg_bleu2, avg_bleu3, avg_bleu4, args.log_file)
        # print(f"Epoch {epoch} - Loss: {avg_loss:.6f}, BLEU-1: {avg_bleu1:.4f}, BLEU-2: {avg_bleu2:.4f}, "
        #       f"BLEU-3: {avg_bleu3:.4f}, BLEU-4: {avg_bleu4:.4f}")

    if epoch%3==0 or epoch==6: #== 0:
        torch.save(embeds_shared.state_dict(), os.path.join(args.save_model_path, 'embeds_shared_epoch{}.pth'.format(epoch)))
        torch.save(encoder.state_dict(), os.path.join(args.save_model_path, 'encoder_epoch{}.pth'.format(epoch)))
        torch.save(decoder.state_dict(), os.path.join(args.save_model_path, 'decoder_epoch{}.pth'.format(epoch)))
        print("Epoch {} model saved!".format(epoch + 1))


def train_TwoAgents(encoder, decoder, device, train_loader, optimizer_encoder, optimizer_decoder, epoch):
    global opt_decoder
    # if data_parallel: torch.cuda.synchronize()

    print('--------------------epoch: %d' % epoch)

    for batch_idx, (train_sents, len_batch) in enumerate(train_loader):
        # training decoder.
        if opt_decoder:
            print('training decoder.')
            decoder.train()
            with torch.no_grad():  # when training decoder, we fix the encoder and make it deterministic, i.e., std=0
                encoder.eval()
                train_sents = train_sents.to(device)  # with eos
                output_float, src_mask = encoder(train_sents, len_batch)
                output_float = normlize_layer.apply(output_float)
                output = getattr(channel, args.channel_type)(output_float, _snr=args.snr)

            output = smaple_n_times(args.multiple_sample, output)
            # decoder sample with softmax policy
            sample_captions, sample_logprobs, seq_masks = decoder.forward_rl(output, sample_max=False)

            fns = list(range(sample_logprobs.size()[0]))
            reward_decoder, cider_mean_decoder = get_self_critical_reward_newsc_TXRL(
                sample_captions, fns, train_sents, 1, 2, reward_scorer)
            loss_decoder = crit('rl', sample_logprobs, seq_masks, reward_decoder) / args.accumulate_grad

            loss_decoder.backward()
            if (batch_idx + 1) % args.accumulate_grad == 0:
                clip_gradient(optimizer_decoder, 0.1)
                optimizer_decoder.step()
                optimizer_decoder.zero_grad()
                opt_decoder = (opt_decoder + 1) % 2  # switch the training flag

        else:
            print('now we optimizer encoder.')
            # now we optimizer encoder.
            encoder.train()
            train_sents = train_sents.to(device)
            output_float_raw, src_mask = encoder(train_sents, len_batch)
            output_float = normlize_layer.apply(output_float_raw)
            output_float = smaple_n_times(args.multiple_sample, output_float)  # get advantage.
            #  when training encoder, we sample with Gaussian policy
            output_sampled, logprobs = policy.forward_sample(output_float, std=0.1)
            output_sampled = normlize_layer.apply(output_sampled)
            output = getattr(channel, args.channel_type)(output_sampled, _snr=args.snr)

            with torch.no_grad():
                # here we fix the decoder and make it deterministic, i.e., `sample_max=True`
                decoder.eval()
                sample_captions, sample_logprobs, seq_masks = decoder.forward_rl(output, sample_max=True, multiple_sample=1)
                fns = list(range(sample_logprobs.size()[0]))
                reward_encoder, cider_mean_encoder = get_self_critical_reward_newsc_TXRL(  # all is tensor type.
                    sample_captions, fns, train_sents, 1, 2, reward_scorer)
            loss_encoder = crit('tx_gaussian_sample', logprobs, reward_encoder) / args.accumulate_grad

            loss_encoder.backward()
            if (batch_idx + 1) % args.accumulate_grad == 0:
                clip_gradient(optimizer_encoder, 0.1)
                optimizer_encoder.step()
                optimizer_encoder.zero_grad()
                opt_decoder = (opt_decoder + 1) % 2

        if batch_idx % 100 == 0 and opt_decoder == 0:
            # for test only. In this case, both encoder and decoder are deterministic.
            # i.e., (encoder: std=0; decoder: argmax)
            with torch.no_grad():
                output_test = getattr(channel, args.channel_type)(output_float.view(
                                                train_sents.shape[0], args.multiple_sample, -1)[:, 0, :],
                                                _snr=args.snr)
                sample_captions, sample_logprobs, seq_masks = decoder.forward_rl(
                                                output_test, sample_max=True, multiple_sample=1)
                fns = list(range(sample_logprobs.size()[0]))
                _, cider_mean_test = get_self_critical_reward_newsc_TXRL(
                    sample_captions, fns, train_sents, 1, 2, reward_scorer)
            print('[%4d / %4d]    ' % (batch_idx, epoch), ' cider_mean_decoder =%.4f,   loss_decoder = %.4f,  '
                                                          'cider_mean_encoder =%.4f,   loss_encoder = %.4f,   now_cider = %.4f'
                  % (float(cider_mean_decoder), loss_decoder.item(), float(cider_mean_encoder),
                     loss_encoder.item(), float(cider_mean_test)))

    if epoch % 3 == 0:  # == 0:
        torch.save(embeds_shared.state_dict(), os.path.join(args.save_model_path, 'embeds_shared_epoch{}.pth'.format(epoch)))
        torch.save(encoder.state_dict(),
                   os.path.join(args.save_model_path, 'encoder_epoch{}.pth'.format(epoch)))
        torch.save(decoder.state_dict(),
                   os.path.join(args.save_model_path, 'decoder_epoch{}.pth'.format(epoch)))
        print("Epoch {} model saved!".format(epoch + 1))



# start training
print('[*]---------------Start training.')
for epoch in range(args.max_epoch):
    if epoch >= args.ckpt_resume-1:
        if args.SemanticRL_JSCC:
            train(encoder, decoder, device, train_data_loader, optimizer, epoch)
        else:
            train_TwoAgents(encoder, decoder, device, train_data_loader, optimizer_encoder, optimizer_decoder, epoch)
    else:
        print('skipping epoch ', epoch)
    if args.SemanticRL_JSCC:
        scheduler.step()
    else:  # variant SCSIU
        scheduler_encoder.step()
        scheduler_decoder.step()

Overwriting /kaggle/working/SemanticRL/Trainng_SemanticRL.py


### Fine tuning

In [17]:
# AWGN-CE-Stage2
!python3 Trainng_SemanticRL.py --training_config /kaggle/working/SemanticRL/config/config_AWGN_CE_Stage2.yaml --dataset_path /kaggle/working/SemanticRL/dataset --log_bleu


[*]---------------args init done
args preview:
snr: 10
multiple_sample: 5
scheduled_sampling_start: 18
iscomplex: 1
channel_type: awgn
batch_size: 64
channel_dim: 256
ckpt_resume: 25
lr_milestones: 160
max_epoch: 75
seeds: 7
init_learning_rate: 0.0001
save_model_path: ./ckpt_AWGN_CE_Stage2/
dataset_path: /kaggle/working/SemanticRL/dataset
RL_training: 0
reward_type: CIDEr
device: cuda:0
backbone: LSTM
training_config: /kaggle/working/SemanticRL/config/config_AWGN_CE_Stage2.yaml
teacher_forcing: 0
SemanticRL_JSCC: 1
accumulate_grad: 20
log_bleu: True
log_file: ./results/training_log.txt
[*]---------------starting preparing dataset and network. Preparing the dataset may take a few minutes
[*]------------vocabulary size is:---- 24064
[*]------------sentences size is:---- 709588
Model LSTM have 4662272 paramerters in total
[*]---------------loaded ckpt at./ckpt_AWGN_CE_Stage2/
/usr/local/lib/python3.11/dist-packages/torch/_compile.py:32: UserWarning: optimizer contains a parameter group w

### Testing

In [10]:
%%writefile /kaggle/working/SemanticRL/Evaluation/Run_Inference_Checkpoints.py
"""
This work is created by KunLu. Copyright reserved.
lukun199@gmail.com
19th Feb., 2021

# Inference.py
"""
import os, platform, json, time, argparse, random, sys
sys.path.append('./')
import torch
from math import log
from data_loader import Dataset_sentence_test, collate_func
from model import LSTMEncoder, LSTMDecoder, Embeds
from utils import Normlize_tx, Channel, smaple_n_times
import torch.utils.data as data
from functools import reduce



def onetime_sample_max(ckpt_this, input_data, encoder, decoder, normlize_layer, channel, len_batch, save_dir, eval_times=1):
    result_strs = []
    gt_strs = []
    to_save_dict = {}
    minib = 2048
    input_data_list = input_data.split(minib)
    len_batch_list = len_batch.split(minib)
    out_chunk = []
    gt_chunk = []
    for idxx, train_sent in enumerate(input_data_list):
        train_sent = train_sent[:,1:]
        train_sent, train_len = collate_func((zip(train_sent, len_batch_list[idxx]-1)))  # len minus 1 for LSTM

        with torch.no_grad():
            output, _ = encoder(train_sent, train_len)
            output = normlize_layer.apply(output)
            output = smaple_n_times(eval_times, output)
            if args.channel == 'gaussian':
                output = channel.awgn(output, _snr=_snr)
            elif args.channel == 'fading':
                output = channel.phase_invariant_fading(output, _snr=_snr)
            elif args.channel == 'vary_gaussian':
                output = channel.awgn(output, _snr=_snr + random.uniform(-10, 10))
            else:
                raise NotImplementedError
            
            output = decoder.sample_max_batch(output, x_mask=None)

        out_chunk.append(output.cpu().numpy().tolist())
        gt_chunk.append(smaple_n_times(eval_times, train_sent).cpu().numpy().tolist())

        if idxx % 4 ==0: print('processing {}'.format(idxx*minib))

    for xx in range(len(out_chunk)):
        for x in range(len(out_chunk[xx])):
            gt_strs.append(gt_chunk[xx][x])
            result_strs.append(out_chunk[xx][x])

    to_save_dict['gt_strs'] = gt_strs
    to_save_dict['result_strs'] = result_strs

    json.dump(to_save_dict, open(save_dir + '/ckpt{}.json'.format(ckpt_this), 'w'))




if __name__ == "__main__":

    _snr = 10
    _iscomplex = True
    channel_dim = 256

    parser = argparse.ArgumentParser()
    parser.add_argument("--path", type=str, default='./ckpt_AWGN_RL')
    parser.add_argument("--name", type=str, default='Test_AWGN_RL')
    parser.add_argument("--data_root", type=str, default='H:\MASTER')
    parser.add_argument("--epoch_start", type=int, default=72)
    parser.add_argument("--epoch_end", type=int, default=73)
    parser.add_argument("--channel", type=str, default='gaussian')
    args = parser.parse_args()

    data_path = os.path.join(args.data_root)
    data_train = Dataset_sentence_test(_path=data_path)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # use CPU or GPU
    embeds_shared = Embeds(vocab_size=data_train.get_dict_len(), num_hidden=128).to(device)
    encoder = LSTMEncoder(channel_dim=channel_dim, embedds=embeds_shared).to(device)
    decoder = LSTMDecoder(channel_dim=channel_dim, embedds=embeds_shared, vocab_size=data_train.get_dict_len()).to(
        device)

    encoder = encoder.eval()
    decoder = decoder.eval()
    embeds_shared = embeds_shared.eval()

    normlize_layer = Normlize_tx(_iscomplex=_iscomplex)
    channel = Channel(_iscomplex=_iscomplex)


    #for rl training
    ckpt_dir = args.path
    save_dir = './Evaluation/InferenceResutls/{}'.format(args.name)
    if not os.path.exists(save_dir): os.makedirs(save_dir)

    all_data = torch.tensor(data_train.data_num).to(device)
    len_batch = torch.tensor(list(map(lambda s: sum(s != 0), all_data))).to(device)

    ss = time.time()

    for idx in range(args.epoch_start, args.epoch_end):

        if idx%3 ==0 and not os.path.exists(save_dir + '/ckpt{}.json'.format(idx)):

            ckpt_this = idx

            print('processing epoch {}'.format(ckpt_this))
            model_path = '_epoch{}.pth'.format(ckpt_this)

            encoder.load_state_dict(torch.load(ckpt_dir + '/encoder' + model_path))##,  map_location='cpu'))
            decoder.load_state_dict(torch.load(ckpt_dir + '/decoder' + model_path))#,  map_location='cpu'))
            embeds_shared.load_state_dict(torch.load(ckpt_dir + '/embeds_shared' + model_path))#,  map_location='cpu'))

            onetime_sample_max(ckpt_this, all_data, encoder, decoder, normlize_layer, channel, len_batch, save_dir)

            print('total time cost at {}: '.format(ckpt_this), time.time()-ss)
        else:
            print('skipping ckpt', idx)

    print('done_all!')


Overwriting /kaggle/working/SemanticRL/Evaluation/Run_Inference_Checkpoints.py


In [ ]:
# !python3 /kaggle/working/SemanticRL/Evaluation/Run_Inference_Checkpoints.py --path /kaggle/working/SemanticRL/ckpt_AWGN_CE_Stage2 --name bleunotRL --data_root /kaggle/working/SemanticRL/dataset

### Estimate total time of running

In [17]:
!python3 /kaggle/working/SemanticRL/Evaluation/Run_Inference_Checkpoints.py --path /kaggle/input/model-bleu-notrl/kaggle/working/SemanticRL/ckpt_AWGN_CE_Stage2 --name bleunotRL --data_root /kaggle/working/SemanticRL/dataset

[*]------------vocabulary size is:---- 24064
[*]------------sentences size is:---- 177398
processing epoch 72
processing 0
processing 8192
processing 16384
processing 24576
processing 32768
processing 40960
processing 49152
processing 57344
processing 65536
processing 73728
processing 81920
processing 90112
processing 98304
processing 106496
processing 114688
processing 122880
processing 131072
processing 139264
processing 147456
processing 155648
processing 163840
processing 172032
total time cost at 72:  227.89959526062012
done_all!


In [18]:
%%writefile /kaggle/working/SemanticRL/Evaluation/CaptionMetrics-master/Eval_Metric_Checkpoints.py
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.spice.spice import Spice
import json, os, argparse



def proc_and_to_list(input):
    res = []
    tmp = ''
    for x in input:
        if x != 0 and x!= 2:
            tmp+=(' '+str(x))
    res.append(tmp[1:])
    return res


def cal_fun():
    """specify the path of json file, then calculate the metrics."""

    save_path = './Evaluation/EvalResults/{}'.format(args.name)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for x in range(args.epoch_start,args.epoch_end):
        if os.path.exists('{}/ckpt{}.json'.format(args.path, x)) \
                and not os.path.exists(save_path + '/scores_ckpt{}.json'.format(x)):
            score_dict = {}
            score_bleu_dict, score_cider_dict, score_rouge_dict = {}, {}, {}

            with open('{}/ckpt{}.json'.format(args.path, x), 'rb') as file:
                tmp = json.load(file)
                gts, res = {idx: proc_and_to_list(cont) for idx, cont in enumerate(tmp['gt_strs'])}, \
                           {idx: proc_and_to_list(cont) for idx, cont in enumerate(tmp['result_strs'])}
            score_bleu, score_cider, score_rouge = main(gts, res)
            score_bleu_dict[x] = score_bleu
            score_cider_dict[x] = score_cider
            score_rouge_dict[x] = score_rouge

            score_dict['bleu'] = score_bleu_dict
            score_dict['cider'] = score_cider_dict
            score_dict['rouge'] = score_rouge_dict

            json.dump(score_dict, open(save_path + '/scores_ckpt{}.json'.format(x), 'w'))
        else:
            print('-----[*]-----skipping ckpt at ', x)


def bleu(gts, res):
    scorer = Bleu(n=4)
    score, scores = scorer.compute_score(gts, res)
    print('belu = %s' % score)
    return score

def cider(gts, res):
    scorer = Cider()
    # scorer += (hypo[0], ref1)
    (score, scores) = scorer.compute_score(gts, res)
    print('cider = %s' % score)
    return score

def meteor(gts, res):
    scorer = Meteor()
    score, scores = scorer.compute_score(gts, res)
    print('meter = %s' % score)
    return score

def rouge(gts, res):
    scorer = Rouge()
    score, scores = scorer.compute_score(gts, res)
    print('rouge = %s' % score)
    return score

def spice(gts, res):
    scorer = Spice()
    score, scores = scorer.compute_score(gts, res)
    print('spice = %s' % score)
    return score

def main(gts, res):
    score_bleu = bleu(gts, res)
    score_cider = cider(gts, res)
    #meteor(gts, res)
    score_rouge = rouge(gts, res)
    #spice(gts, res)
    return score_bleu, score_cider, score_rouge

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("--path", type=str, default='')
    parser.add_argument("--name", type=str, default='')
    parser.add_argument("--epoch_start", type=int, default=72)
    parser.add_argument("--epoch_end", type=int, default=73)
    args = parser.parse_args()
    cal_fun()

Overwriting /kaggle/working/SemanticRL/Evaluation/CaptionMetrics-master/Eval_Metric_Checkpoints.py


In [19]:
!python3 /kaggle/working/SemanticRL/Evaluation/CaptionMetrics-master/Eval_Metric_Checkpoints.py --path /kaggle/working/SemanticRL/Evaluation/InferenceResutls/bleunotRL --name ckpt72.json

{'testlen': 2349698, 'reflen': 2324783, 'guess': [2349698, 2172300, 1994902, 1817504], 'correct': [8929, 111, 2, 0]}
ratio: 1.0107171292976587
belu = [0.003800062816583236, 0.0004406532485714403, 5.795631898771724e-05, 3.2170421546079057e-09]
cider = 0.009078183307135999
rouge = 0.00532383406820812


In [29]:
%%writefile /kaggle/working/SemanticRL/Evaluation/Inference_Given_Input.py
"""
This work is created by KunLu. Copyright reserved.
lukun199@gmail.com
19th Feb., 2021

# Inference.py
"""
import os, platform, json, time, pickle, sys, argparse
import torch
from math import log
sys.path.append('./')
from data_loader import Dataset_sentence_test, collate_func
from model import LSTMEncoder, LSTMDecoder, Embeds
from utils import Normlize_tx, Channel, smaple_n_times


_snr = 10
_iscomplex = True
channel_dim = 256


device = torch.device("cpu:0")
#torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # use CPU or GPU
embeds_shared = Embeds(vocab_size=24064, num_hidden=128).to(device)
encoder = LSTMEncoder(channel_dim=channel_dim, embedds=embeds_shared).to(device)
decoder = LSTMDecoder(channel_dim=channel_dim, embedds=embeds_shared, vocab_size = 24064).to(device)

encoder = encoder.eval()
decoder = decoder.eval()
embeds_shared = embeds_shared.eval()


normlize_layer = Normlize_tx(_iscomplex=_iscomplex)
channel = Channel(_iscomplex=_iscomplex)


def do_test(input_data, encoder, decoder, normlize_layer, channel, len_batch):

    with torch.no_grad():
        output, _ = encoder(input_data, len_batch)
        output = normlize_layer.apply(output)
        output = channel.awgn(output, _snr=_snr)
        output = decoder.sample_max_batch(output, None)

    return output

SemanticRL_example = ['this is a typical sentence used to check the performance',
                        'this is a typical unk used to check the performance',
                      'this is exactly a long sentence with complex structure which might be a challenge for both',
                       'i have just brought a yellow banana',
                      'a man is holding a giant elephant on his hand',
                      ]

if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument("--ckpt_pathCE", type=str, default='./ckpt_AWGN_CE_Stage2')
    parser.add_argument("--ckpt_pathRL", type=str, default='./ckpt_AWGN_RL_SemanticRLv1')  # or './ckpt_AWGN_RL'
    args = parser.parse_args()

    dict_train = pickle.load(open('./train_dict.pkl', 'rb'))
    rev_dict = {vv: kk for kk, vv in dict_train.items()}
    
    for input_str in SemanticRL_example:

        input_vector = [dict_train[x] for x in input_str.split(' ')] + [2]
        input_len = len(input_vector)
        input_vector = torch.tensor(input_vector)
    
        ckpt_dir = args.ckpt_pathCE
        model_name = os.path.basename(ckpt_dir)
    
        encoder.load_state_dict(torch.load(ckpt_dir + '/encoder_epoch72.pth', map_location='cpu'))
        decoder.load_state_dict(torch.load(ckpt_dir + '/decoder_epoch72.pth', map_location='cpu'))
        embeds_shared.load_state_dict(torch.load(ckpt_dir + '/embeds_shared_epoch72.pth',  map_location='cpu'))
    
        for _ in range(5):
            output = do_test(input_vector.unsqueeze(0), encoder, decoder, normlize_layer, channel,
                    len_batch=torch.tensor(input_len).view(-1, ))
            output = output.cpu().numpy()[0]
            res = ' '.join(rev_dict[x] for x in output if x!=0 and x!=2)  # remove 'PAD' and 'EOS'
            print('result of {}:            {}'.format(model_name, res))
        print('--------------------------------------------------')

Overwriting /kaggle/working/SemanticRL/Evaluation/Inference_Given_Input.py


In [30]:
!python3 /kaggle/working/SemanticRL/Evaluation/Inference_Given_Input.py

result of ckpt_AWGN_CE_Stage2:            this is a typical mep again to check the performance
result of ckpt_AWGN_CE_Stage2:            this is a typical sentence opportunity to check the performance
result of ckpt_AWGN_CE_Stage2:            this is a typical sentence opportunity to check the performance
result of ckpt_AWGN_CE_Stage2:            this is a typical sentence opportunity to check the performance
result of ckpt_AWGN_CE_Stage2:            this is a typical sentence opportunity to check the performance
--------------------------------------------------
result of ckpt_AWGN_CE_Stage2:            this is a typical to continue to check the performance
result of ckpt_AWGN_CE_Stage2:            this is a typical success debate to check the performance
result of ckpt_AWGN_CE_Stage2:            this is a typical order is to check the performance
result of ckpt_AWGN_CE_Stage2:            this is a typical in cases to check the performance
result of ckpt_AWGN_CE_Stage2:            thi

### Training SemanticRL-SCSIU

In [ ]:
!python3 /kaggle/working/SemanticRL/Trainng_SemanticRL.py --training_config /kaggle/working/SemanticRL/config/config_AWGN_RL_SCSIU.yaml --dataset_path /kaggle/working/SemanticRL/dataset